In [1]:
from urllib.request import Request, urlopen, HTTPError
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
import pandas as pd
import csv

keyword = ['데이터 사이언티스트', '데이터애널리스트', '데이터 엔지니어']
page = 1
front_url = 'https://www.saramin.co.kr/zf_user/search/recruit?search_area=main&search_done=y&search_optional_item=n&searchType=recently&searchword='
mid_url = '&recruitPage='
back_url = '&recruitSort=relation&recruitPageCount=100&&inner_com_type=scale001&company_cd=0%2C1%2C2%2C3%2C4%2C5%2C6%2C7%2C9%2C10&show_applied=&quick_apply=&except_read=&ai_head_hunting='
url_data = []
company_name = []
position_name = []
post_title = []
company_position = []
content = []

chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# url, 공고 이름, 기업명, 기업 주소를 찾는 크롤링코드
for search_keyword in keyword:
    driver.get(front_url+search_keyword+mid_url+str(page)+back_url)
    ct_result = driver.find_element(By.XPATH, '//*[@id="recruit_info"]/div[1]/span').text
    total_result = re.sub(r'[^0-9]', '', ct_result)
    page_ct = int(int(total_result)/100)

    for pages in range(1, page_ct+2):
        try:
            driver.get(front_url+search_keyword+mid_url+str(pages)+back_url)
            driver.implicitly_wait(time_to_wait=60)

            div_1 = driver.find_elements(By.CLASS_NAME, "job_tit")
            for item in div_1:
                tmp_url = item.find_element(By.TAG_NAME, "a").get_attribute('href')
                url_data.append(tmp_url)
                tmp_title = item.find_element(By.TAG_NAME, "a").text
                post_title.append(tmp_title)

            div_2 = driver.find_elements(By.CLASS_NAME, "corp_name")
            for item in div_2:
                tmp_cn = item.find_element(By.TAG_NAME, "a").text
                company_name.append(tmp_cn)
                position_name.append(search_keyword)

            time.sleep(3)

        except Exception as e:
            pass

driver.close()
time.sleep(3)

driver = webdriver.Chrome("chromedriver")
#기업 주소, 공고내용 전체 크롤링 코드
for url in url_data:
    driver.get(url)
    driver.implicitly_wait(time_to_wait=60)

    try:
        driver.switch_to.frame("iframe_content_0")
        contents = driver.find_element(By.CLASS_NAME, "user_content").text
        content.append(contents)
        
    except:
        content.append('None')
        continue
        
driver.close()

major_df = pd.DataFrame({'회사명' : company_name, '직무' : position_name, '링크' : url_data, '공고내용' : content})
major_df.to_csv("major_df.csv", index = False)

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.30M/6.30M [00:00<00:00, 9.60MB/s]
